In [9]:
import pandas as pd
import matplotlib.pyplot as plt
def importing(link):
    df=pd.DataFrame.from_csv(link,sep='\t')
    PARTICIPANT_A=df.index
    PARTICIPANT_A=pd.DataFrame(PARTICIPANT_A)
    df.index=range(len(df.index))
    df['PARTICIPANT_A']=PARTICIPANT_A
    df.index=df.index.astype(str)
    df=df.T.drop(['INTERACTION_PUBMED_ID','INTERACTION_DATA_SOURCE'])
    df=df.T
    return df

def importing2(link):
    df=pd.DataFrame.from_csv(link,sep='\t')
    df.PATHWAY_NAMES=df.INTERACTION_PUBMED_ID
    PARTICIPANT_A=df.index
    PARTICIPANT_A=pd.DataFrame(PARTICIPANT_A)
    df.index=range(len(df.index))
    df['PARTICIPANT_A']=PARTICIPANT_A
    df.index=df.index.astype(str)
    df=df.T.drop(['INTERACTION_PUBMED_ID','INTERACTION_DATA_SOURCE'])
    df=df.T
    return df

def filt_genes(i,lst,db):
    x=db[db.PARTICIPANT_A==lst[i]]
    y=db[db.PARTICIPANT_B==lst[i]]
    x=pd.concat([x,y])
    return x

def filt_only_genes(i,lst,db):
    x=db[db.PARTICIPANT_A==lst[i]]
    y=db[db.PARTICIPANT_B==lst[i]]
    x=pd.concat([x,y])
    y=pd.concat([x[x.PARTICIPANT_B==j] for j in lst if j!=lst[i]])
    return y

def fltr_togetonlygenes(df):
    def fnc(i):
        df.PARTICIPANT_B[i]='NaN'
    def fnc1(i):
        df.PARTICIPANT_A[i]='NaN'
    [fnc(i) for i in range(len(df.PARTICIPANT_B)) if df.PARTICIPANT_B[i].isupper()!=True]
    [fnc1(i) for i in range(len(df.PARTICIPANT_A)) if df.PARTICIPANT_A[i].isupper()!=True]
    df=df[df.PARTICIPANT_A!='NaN']
    df=df[df.PARTICIPANT_B!='NaN']
    return df

def sep_pths(i):
    x=result.PATHWAY_NAMES[i].index(';')
    result.PATHWAY_NAMES[i][:x]
    result.PATHWAY_NAMES[i][x+1:]
    y=result.irow(i)
    y.PATHWAY_NAMES=result.PATHWAY_NAMES[i][x+1:]
    result.irow(i).PATHWAY_NAMES=result.PATHWAY_NAMES[i][:8]
    result.loc[str(int(result.index[-1])+1)]=y
    


In [10]:
kegg=importing('/cellar/users/mkose/kegg.pathways.tmp')   # importing pathway data from different sources
biogrid=importing2('/cellar/users/mkose/Downloads/interactions/Pathway Commons.7.BioGRID.EXTENDED_BINARY_SIF.hgnc.sif')
hprd=importing2('/cellar/users/mkose/Downloads/interactions/Pathway Commons.7.HPRD.EXTENDED_BINARY_SIF.hgnc.sif')
intact=importing2('/cellar/users/mkose/Downloads/interactions/Pathway Commons.7.IntAct.EXTENDED_BINARY_SIF.hgnc.sif')
panther=importing('/cellar/users/mkose/Downloads/interactions/Pathway Commons.7.PANTHER Pathway.EXTENDED_BINARY_SIF.hgnc.sif')

In [11]:
gene_list1=['PTK2','VASP','TLN2','ZYX']  #gene lists we obtained from RNA-seq analysis.
gene_list2=['ACTN1','PTK2','RAC1','TLN1','TLN2','VCL','ZYX']

db_all=pd.concat([kegg,biogrid,hprd,intact,panther]) # getting all pathway data into one data frame.
db_all=db_all.drop_duplicates()

In [12]:
result=pd.concat([filt_genes(i,gene_list2,db_all) for i in range(len(gene_list2))]) # taking the rows which include genes in our list.
result=result.drop_duplicates()
result=result[~result.PATHWAY_NAMES.isnull()]
[sep_pths(i) for i in range(len(result)) if ';' in result.PATHWAY_NAMES[i]]
i

20293

In [13]:
pthwys=result.PATHWAY_NAMES.drop_duplicates() #the list of pathway names(or numbers)
counts={} # creating the matris which has the names of pathways and the number of genes involved in this pathways.
[counts.update({i:len([1 for j in range(len(gene_list2)) if True in list(result[result.PATHWAY_NAMES==pthwys[i]].PARTICIPANT_B==gene_list2[j]) or True in list(result[result.PATHWAY_NAMES==pthwys[i]].PARTICIPANT_A==gene_list2[j])])
}) for i in range(len(pthwys))] # 
counts=pd.DataFrame(counts,index=['counts'])
counts=counts.T
counts.index=pthwys.values 
counts=counts.drop('BIOGRID:')
counts=counts.sort('counts')
pathway_data=result[result.PATHWAY_NAMES==counts.index[-1]] #taking the rows which include the patway that has most number of genes.
#pathway_data=pathway_data[pathway_data.INTERACTION_TYPE=='interacts-with']


In [6]:
# removing the genes which are not in the pathway
[ gene_list2.remove(i) for i in gene_list2 if not i in list(pathway_data.PARTICIPANT_A) and not i in list(pathway_data.PARTICIPANT_B)]

[None]

In [7]:
# visulization of the pathway for Cytoscape
from py2cytoscape.data.cyrest_client import CyRestClient
from IPython.display import display
from IPython.display import Image
import networkx as nx 
cy = CyRestClient()

cy.session.delete() #here we deleting current sessions so be careful if you have a session you working with delete here or save it.

G=nx.Graph() #we are creating our network
[G.add_edge(pathway_data.PARTICIPANT_A[i],pathway_data.PARTICIPANT_B[i]) for i in range(len(pathway_data))]#adding nodes and edges

network=cy.network.create_from_networkx(G)#adding network to cytoscape
cy.layout.apply(name='kamada-kawai', network=network)
kv_pair = {} 
[kv_pair.update({i:'blue'}) for i in gene_list2] #assigning blue color to the genes we want
style = cy.style.create('Nested Network Style')
style.create_discrete_mapping(column='name',col_type='String',vp='NODE_FILL_COLOR', mappings=kv_pair)#changeing the color of the genes we want.

cy.style.apply(style, network=network)
result = cy.edgebundling.apply(network)
network_png = network.get_png()
Image(network_png) 

ConnectionError: ('Connection aborted.', error(111, 'Connection refused'))

In [8]:
# visulization of the pathway for networkx
G=nx.Graph()#we are creating our network
[G.add_edge(pathway_data.PARTICIPANT_A[i],pathway_data.PARTICIPANT_B[i]) for i in range(len(pathway_data))] #adding nodes and edges
pos=nx.spring_layout(G)
y=G.nodes()
[y.remove(gene_list2[i]) for  i in range(len(gene_list2))]#removing the genes in the list
nx.draw_networkx_nodes(G,pos,nodelist=y,node_color='y') #assigning color to the other genes 
nx.draw_networkx_nodes(G,pos,nodelist=gene_list2,node_color='b')#assignig color to the genes in the list
nx.draw_networkx_edges(G,pos)
nx.draw_networkx_labels(G,pos)
plt.show()


In [33]:
#pathway_data.to_csv('/cellar/users/mkose/pathway_data.csv')